# How to Find the Max Precipitation Value of an ROI Using an ArcGIS Image Service

### Overview
The goal of this tutorial is to inform the user on how to find the maximum precipitation rate (mm/hr) using an ArcGIS image service as data input, via a Jupyter Notebook (JN) for a defined Region of Interest (ROI). The ArcGIS image service used in the notebook is the "[GPM_3IMERGHHE_06 (ImageServer)](https://arcgis.gesdisc.eosdis.nasa.gov/authoritative/rest/services/GPM_3IMERGHHE_06/ImageServer)" ArcGIS REST service [(Huffman et. al., 2019)](https://disc.gsfc.nasa.gov/datasets/GPM_3IMERGHH_06/summary).

### Prerequisites

1. A Python3 coding environment, such as an updated, local instantce of ESRI ArcGIS Pro (desktop), Visual Studio Code/VS Code (desktop), ESRI ArcGIS Enterprise (web), Jupyter Notebook (web), Google Colaboratory (web), a shell/terminal/command-line such as IPython in the command window (desktop), and several other options. Though many interfaces can be used, the code has been tested in the aforementioned environments. This notebook was written using Python 3.9, and requires these libraries and files: [tornado](https://pypi.org/project/tornado/) and [arcgis](https://developers.arcgis.com/python/).   

2. This notebook requires having the latest versions of the "arcgis" python API and "datetime" python module. If running the JN in Google Colaboratory, you will also need to have the latest version of the "tornado" Python web framework. If you don't have these versions, you can install them using the instructions in Step 3 (below) in this prerequisites section. Some environments (e.g. - ESRI ArcGIS Pro) are packaged automatically with the arcgis library, and don't require a separate installation. If you are unsure what libraries you are require to install, run the JN with the code as-is in the desired environment. A runtime error should appear and specify which Python modules it cannot access and need to be installed. If no errors appear, then you are all set to proceed.

3. Methods to install these libraries can vary, depending on your coding environment. You will need to use the conda, and/or pip packaging tools, to install the required Python libraries to your environment. You can use the optional lines of code in the first cell of the JN to install the libraries to your environment, or you can install the needed libraries locally using the Windows command line. For the former, the lines in the JN are currently inactive ("commented-out"), and can be selectively activated based on your unique situation by removing the hastag symbols. For the latter, please see an example of this for use in Visual Studio Code in the "Additional Info" section. Also note, if you are using Anaconda as your default Python installation, you will need to use both "pip" and "conda" packaging tools to install the arcgis library.

### 1. Environment Setup and Importing Libraries

1. Download the Jupyter Notebook file attached to this How-to and save it to your computer in an accessible folder (it will have a “.ipynb” file extension). 

2. Open your .ipynb file in the desired interface/environment. 

3. Please see the "Prerequisites" section before proceeding. If you already have the required libraries installed, be sure they are the latest versions; if not, update them.

4. Once you have completed the above steps, run the first cell of the JN to import the modules for the subsequent cells:

In [ ]:
      #Required for Google Colaboratory use only - Install tornado and arcgis
#!pip install tornado --upgrade
#import tornado.ioloop #necessary import
#import tornado.web #necessary import
#!pip install arcgis

#--------

    #Install arcgis using Anaconda environment (non-Google Colab) - set your shell for first-time use
#conda init <shell name> #set your shell (e.g. - powershell, cmd.exe, bash, etc.) --> IMPORTANT: After running conda init, you may need to restart your shell for changes to take effect
#conda activate base
#conda install -c esri arcgis

#--------

      #Install arcgis using pip (non-Google colab)
#pip install arcgis

#--------

      #MAY need to use both pip and conda if Anaconda is your default Python installtion - see Additional Info section in the how-to for an example

#--------

#Import modules - needed for all environments
import arcgis
from arcgis.raster import *
import datetime
from datetime import timezone


print("Done installing and importing python packages.")

### 2. Convert Times

The second cell of the notebook converts the time input to epoch format.  Running the code as-is will print the included “start” and “end” epochs, which are based on the input UTC times. The UTC times can be altered by the user. You can enter your desired “Start” and “End” Year, Month, Day and Hour to define your own temporal range, by replacing the values in the code excerpt shown below from the first cell of the JN. Once the values have been changed, you can run the code again to print the new times in epoch format that will be used as input for the next cell. 

In [2]:
#User-defined start time in UTC
start_year = 2022
start_month = 5
start_day = 17
start_hour = 0

#User-defined end time in UTC
end_year = 2022
end_month = 5
end_day = 22
end_hour = 0

#UTC times from user-input
start_utc_dt = datetime.datetime(start_year, start_month, start_day, start_hour)
end_utc_dt = datetime.datetime(end_year, end_month, end_day, end_hour)

#utc-timestamp conversion
start_timestamp = int(start_utc_dt.replace(tzinfo=timezone.utc).timestamp())*1000
end_timestamp = int(end_utc_dt.replace(tzinfo=timezone.utc).timestamp())*1000

#print start and end timestamp
print(start_timestamp)
print(end_timestamp)

1652745600000
1653177600000


### 3. Enter Coordinates and Call Imagery Service

1. This notebook also requires latitude and longitude coordinates in Decimal Degrees (DD) format. Keep the values as-is, or enter your desired DD values in the code excerpt shown below. This will define your ROI's geographic bounding box, using the xmax, xmin, ymax, and ymin input coordinates. The “wkid” geographic coordinate system (GCS) is currently set for WGS 1984. This does not need to be changed, since that is the native GCS of the image service. Note that with a larger geographic area (AOI) and longer time range (start to end times), the JN will take longer to run.

2. Once the desired parameters are set, run the third JN cell. It will output your maximum precipitation rate in mm/hr found within the defined ROI and temporal range. Note: it does not output the type of precipitation (rain, snow, sleet, etc.). Your output should look like the following when using the input values from this tutorial (your value will be different if the input times and geographic area are altered): 

`Please wait for histograms to compute over the defined time range and aoi...this could take a few minutes.
23.83  mm/hr is the maximum GPM precipitation value reported between the defined time range within the aoi.`

In [ ]:
#Define data source (NASA GPM IMERGHHE Imagery Service)
data = ImageryLayer('https://gis.earthdata.nasa.gov/image/rest/services/GESDISC/GPM_3IMERGHHE/ImageServer')

#Define lists and time ranges
time_range = []
time_range = range(start_timestamp, end_timestamp, 1800000) #times in epoch milliseconds; 1800000 represents 0.5-hr timestep
max_val_list = []

#Define Area of Interest (aoi)
aoi = {"spatialReference": {"wkid": 4326},
        "xmax": -71.254028,
        "xmin": -71.50000,
        "ymax": 46.829853,
        "ymin": 46.500000}

#Log message
print("Please wait for histograms to compute over the defined time range and aoi...this could take a few minutes.")

#Loop through computing histograms for each time slice over a constant aoi; extract each slice histogram's 'max' value and add to list
for t in time_range:
    comp_histo = data.compute_histograms(geometry=aoi, rendering_rule={"rasterFunction":None}, time=str(t))
    if comp_histo['histograms'] == []:
        max_val = 0
        max_val_list.append(float(max_val))
    else:
        max_val = comp_histo['histograms'][0]['max']
        max_val_list.append(float(max_val))

#Sort the max val list over all time slices and aoi
max_val_list.sort()

#Print the overall maximum GPM value in mm/hr
print(round(max_val_list[-1],2), " mm/hr is the maximum GPM precipitation value reported between the defined time range within the aoi.")

Please wait for histograms to compute over the defined time range and aoi...this could take a few minutes.
23.83  mm/hr is the maximum GPM precipitation value reported between the defined time range within the aoi.


This concludes the necessary steps and explanation of the JN. The [GPM_3IMERGHHE_06 ("Early")](https://arcgis.gesdisc.eosdis.nasa.gov/authoritative/rest/services/GPM_3IMERGHHE_06/ImageServer) precipitation data product used in this JN is a near real-time, global GIS dataset displaying precipitation rate data from June 1st, 2000, through present, and is updated daily with a seven-hour latency. The service has applications in quick-response hazard monitoring, and environmental modeling, analysis, and mapping. It is also intended for quick and efficient user interaction. The service is not intended for use in scientific research.